In [1]:
import json  # Importa il modulo json per lavorare con dati in formato JSON
import requests  # Importa il modulo requests per effettuare richieste HTTP
from openai import OpenAI  # Importa la classe OpenAI dal pacchetto openai
from dotenv import (
    load_dotenv,
)  # Importa la funzione load_dotenv per caricare variabili d'ambiente da un file .env
import os  # Importa il modulo os per interagire con il sistema operativo

# Carica le variabili d'ambiente definite nel file .env
load_dotenv()

# Recupera la chiave API di OpenAI dalla variabile d'ambiente
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# Crea un'istanza del client OpenAI usando la chiave API
client = OpenAI(api_key=OPENAI_API_KEY)

In [ ]:
def get_country_info(country_name: str):
    """
    Recupera informazioni su un paese specifico utilizzando l'API di OpenAI.
    
    Args:
        country_name (str): Il nome del paese di cui si vogliono ottenere informazioni.
    
    Returns:
        dict: Un dizionario contenente le informazioni sul paese: capitale, popolazione, regione geografica (es. Europa, Asia, etc.)
    """

    response = requests.get(
        f"https://restcountries.com/v3.1/name/{country_name}"
    )  # Effettua una richiesta GET all'API REST
    data = response.json()  # Converte la risposta in formato JSON
    # print(data[0].keys())  # Stampa i dati ricevuti per debug
    # print(data[0]["capital"][0])  # Stampa la capitale del paese per debug
    # print(data[0]["population"])  # Stampa la popolazione del paese per debug
    # print(data[0]["region"])  # Stampa la regione del paese per debug
    #return data  # Restituisce il primo elemento della lista di paesi trovati
    return {"country": country_name, "capital": data[0]["capital"][0], "population": data[0]["population"], "region": data[0]["region"]}  # Restituisce un dizionario con le informazioni del paese

In [28]:
# Esempio di utilizzo della funzione get_country_info
country_name = "ITALY"  # Nome del paese da cercare
country_info = get_country_info(country_name)  # Chiama la funzione per ottenere le informazioni sul paese
print(json.dumps(country_info, indent=2))  # Stampa le informazioni del paese in formato JSON con indentazione per una migliore leggibilità

{
  "country": "ITALY",
  "capital": "Rome",
  "population": 59554023,
  "region": "Europe"
}


In [26]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_country_info",
            "description": "Recupera informazioni su un paese specifico.",
            "parameters": {
                "type": "object",
                "properties": {
                    "country_name": {
                        "type": "string",
                        "description": "Il nome del paese di cui si vogliono ottenere informazioni."
                    }
                },
                "required": ["country_name"],
                "additionalProperties": False,  # Non sono ammessi parametri aggiuntivi
            },
            "strict": True,  # Parametri devono essere rispettati rigorosamente
        }
    }
]  # Definisce gli strumenti disponibili per l'API OpenAI, in questo caso la funzione get_country_info

In [29]:
def function_calling(tool_calls):
    for tool_call in tool_calls:  # Itera su tutte le chiamate agli strumenti
        if (tool_call.function.name == "get_country_info"):
            args = json.loads(tool_call.function.arguments)  # Decodifica gli argomenti JSON
            info = get_country_info(args["country_name"])  # Ottiene le informazioni sul paese
            return f"The requested info are:\n" + json.dumps(info, indent=2)  # Restituisce le informazioni in formato JSON con indentazione
        else:
            return "Unknown tool call"  # Gestisce chiamate a strumenti sconosciuti  

In [36]:
response = client.chat.completions.create(
    model="gpt-4.1",
    messages=[
        {
            "role": "system",
            "content": "You are a helpful assistant that provides information about countries.",
        },
        {
            "role": "user",
            "content": f"Tell me about Canada.",
        },
    ],
    tools=tools,
    tool_choice="auto",  # Permette al modello di scegliere automaticamente se usare gli strumenti
    temperature=0.7,  # Imposta la temperatura per la generazione della risposta
)

# Estrae il testo della risposta dal completamento
if response.choices[0].message.tool_calls:
    # Se ci sono chiamate agli strumenti, chiama la funzione function_calling per gestirle
    print(function_calling(response.choices[0].message.tool_calls))
else:
    # Altrimenti, restituisce il contenuto della risposta come stringa
    print(response.choices[0].message.content.strip())


The requested info are:
{
  "country": "Canada",
  "capital": "Ottawa",
  "population": 38005238,
  "region": "Americas"
}
